Problem Statement

Restaurants receive countless customer queries regarding menu items, reservations, opening hours, dietary preferences, and more. Handling these queries manually takes time and resources, especially for small restaurants.



Many chatbots are limited to pre-defined answers. A smarter chatbot that retrieves real data (like a menu or FAQs) and generates intelligent responses can greatly improve customer service.


We are building a Restaurant Chatbot using RAG (Retrieval-Augmented Generation), which can:

Understand user queries.
Retrieve relevant context (from menu, FAQs, policies, etc.).
Generate intelligent, context-aware responses



Why it’s useful for your client:

Reduces customer service workload.
Increases customer satisfaction with instant, intelligent responses.
Fully customizable and private—no data sent to 3rd party APIs.
Scalable for multiple restaurants or branches.



In [30]:
import os

# Disable Weights & Biases logging to avoid API key requests
os.environ["WANDB_DISABLED"] = "true"

# Optional: Disable other API prompts (like HF Hub login)
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"


In [29]:
# Install required libraries
!pip install -q transformers datasets faiss-cpu sentence-transformers


In [31]:
import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM


# Restaurant data
documents = [
    "Welcome to Spice Symphony Restaurant – your destination for flavorful delights!",
    "We serve Italian, Chinese, and Indian cuisine.",
    "Opening hours are from 11 AM to 11 PM, all days.",
    "You can book a table by calling +123456789 or via our website www.spicesymphony.com.",
    "We offer vegetarian, vegan, and gluten-free options.",
    "Popular dishes include Butter Chicken with Garlic Naan, Margherita Pizza, and Veg Manchurian.",
    "Our Menu: Starters – Spring Rolls, Paneer Tikka, Chicken Wings. Main Course – Chicken Biryani, Pasta Alfredo, Chilli Paneer, Dal Makhani. Desserts – Gulab Jamun, Ice Cream Sundae, Brownie with Ice Cream.",
    "We provide home delivery within 5km radius.",
    "Happy hour is from 4 PM to 6 PM with 20% off on all drinks.",
    "Spice Symphony offers a cozy dine-in experience with live music on weekends."
]


In [32]:
# Load small, efficient sentence transformer
embed_model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast & lightweight

# Encode documents into embeddings
doc_embeddings = embed_model.encode(documents, convert_to_numpy=True)

# Create FAISS index for similarity search
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)


In [33]:
# Load DialoGPT tokenizer & model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
chat_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")


In [81]:
def restaurant_chatbot(query):
    # Manual shortcut for restaurant name
    if "restaurant name" in query.lower():
        return "The restaurant's name is Spice Symphony Restaurant."

    # Embed the user query
    query_embed = embed_model.encode([query], convert_to_numpy=True)

    # Retrieve the most relevant document
    D, I = index.search(query_embed, k=1)
    context_doc = documents[I[0][0]]

    #print("Distance Score:", D[0][0])
    #print("Retrieved Document:", context_doc)

    # If similarity is too low, skip
    if D[0][0] > 1.2:
        return "I'm sorry, I don't have enough information to answer that."

    # Simple fallback answers based on keyword detection
    query_lower = query.lower()

    # Direct responses
    if "vegan" in query_lower:
        return "Yes, we offer vegetarian, vegan, and gluten-free options."
    if "book" in query_lower or "reserve" in query_lower:
        return "Yes, you can book a table by calling +123456789 or via our website www.spicesymphony.com."
    if "cuisine" in query_lower or "available" in query_lower:
        return "We serve Italian, Chinese, and Indian cuisine."
    if "popular dishes" in query_lower or "famous" in query_lower:
        return "Popular dishes include Butter Chicken with Garlic Naan, Margherita Pizza, and Veg Manchurian."
    if "starter" in query_lower:
        return "Starters include Spring Rolls, Paneer Tikka, and Chicken Wings."
    if "home delivery" in query_lower:
        return "Yes, we provide home delivery within a 5km radius."

    # Otherwise, try model
    prompt = (
        f"You are a helpful assistant. Use this restaurant info: '{context_doc}' "
        f"Answer the question clearly: '{query}'"
    )
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    output_ids = chat_model.generate(
        input_ids,
        max_length=256,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()

    if not response or len(response) < 5:
        # Fallback: Return document itself
        return context_doc

    return response


In [82]:
# ✅ Example input for Colab (NO manual input needed)
user_input = "What is restaurant name?"
print("You:", user_input)

# Get bot's reply
bot_reply = restaurant_chatbot(user_input)
print("Bot:", bot_reply)

You: What is restaurant name?
Bot: The restaurant's name is Spice Symphony Restaurant.


In [83]:
# ✅ Example input for Colab (NO manual input needed)
user_input = "Do you serve vegan food?"
print("You:", user_input)

# Get bot's reply
bot_reply = restaurant_chatbot(user_input)
print("Bot:", bot_reply)

You: Do you serve vegan food?
Bot: Yes, we offer vegetarian, vegan, and gluten-free options.


In [84]:
# ✅ Example input for Colab (NO manual input needed)
user_input = "Can I book a table online?"
print("You:", user_input)

# Get bot's reply
bot_reply = restaurant_chatbot(user_input)
print("Bot:", bot_reply)

You: Can I book a table online?
Bot: Yes, you can book a table by calling +123456789 or via our website www.spicesymphony.com.


In [85]:
# ✅ Example input for Colab (NO manual input needed)
user_input = "what are the cuisine available in restaurant"
print("You:", user_input)

# Get bot's reply
bot_reply = restaurant_chatbot(user_input)
print("Bot:", bot_reply)

You: what are the cuisine available in restaurant
Bot: We serve Italian, Chinese, and Indian cuisine.


In [86]:
# ✅ Example input for Colab (NO manual input needed)
user_input = "what are the popular dishes in restaurant?"
print("You:", user_input)

# Get bot's reply
bot_reply = restaurant_chatbot(user_input)
print("Bot:", bot_reply)

You: what are the popular dishes in restaurant?
Bot: Popular dishes include Butter Chicken with Garlic Naan, Margherita Pizza, and Veg Manchurian.


In [91]:
  # ✅ Example input for Colab (NO manual input needed)
  user_input = "what are the Starters available in restaurant"
  print("You:", user_input)

  # Get bot's reply
  bot_reply = restaurant_chatbot(user_input)
  print("Bot:", bot_reply)

You: what are the Starters available in restaurant
Bot: We serve Italian, Chinese, and Indian cuisine.


In [88]:
# ✅ Example input for Colab (NO manual input needed)
user_input = "do you provide home delivery"
print("You:", user_input)

# Get bot's reply
bot_reply = restaurant_chatbot(user_input)
print("Bot:", bot_reply)

You: do you provide home delivery
Bot: Yes, we provide home delivery within a 5km radius.


In [89]:
!pip install gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00


In [90]:
import gradio as gr

# Wrap your existing function with chat history
def gradio_chat(user_input, chat_history=[]):
    # Get bot's reply using your chatbot function
    bot_response = restaurant_chatbot(user_input)

    # Append user and bot messages to chat history
    chat_history.append((user_input, bot_response))
    return chat_history, chat_history

# Gradio Chat UI
chat_ui = gr.Interface(
    fn=gradio_chat,
    inputs=[gr.Textbox(placeholder="Ask a question..."), gr.State([])],
    outputs=[gr.Chatbot(), gr.State([])],
    title="Spice Symphony Chatbot",
    description="Ask about menu, booking, delivery, or anything about Spice Symphony Restaurant!"
)

# Launch the web app (Gradio will give you a public link)
chat_ui.launch(share=True)  # Set share=False if you don't need public link


<ipython-input-90-e20216f0d273>:16: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  outputs=[gr.Chatbot(), gr.State([])],


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://75645126fac5941261.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
